<a href="https://colab.research.google.com/github/Vamsi-Malineni/CS5691_Machine_Learning_Models/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recurrent Neural Networks


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sc
import tensorflow as tf
import keras as ks
import random

In [ ]:
random.seed(hash("seriously you compete with me") % 2**32 - 1)
np.random.seed(hash("i am mohammed safi") % 2**32 - 1)
tf.random.set_seed(hash("ur rahman khan") % 2**32 - 1)

In [ ]:
from tensorflow.python.keras import backend as K
K._get_available_gpus()

[]

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!pip install wandb
import wandb

from wandb.keras import WandbCallback

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# English as source language and Telugu as target language
source_language='en'
target_language='te'

# path to the folder where the dataset is located
path = '/content/drive/MyDrive/dakshina_dataset_v1.0/dakshina_dataset_v1.0'

# assigning training data path to the target language
# and reading the tsv file
train_path=os.path.join(path,target_language,"lexicons",target_language+".translit.sampled.train.tsv")
train=pd.read_csv(train_path,sep="\t",names=['tgt','src','count'])

# assigning validation data path to the target language
# and reading the tsv file 
valid_path=os.path.join(path,target_language,"lexicons",target_language+".translit.sampled.dev.tsv")
valid=pd.read_csv(valid_path,sep="\t",names=['tgt','src','count'])

# assigning test data path to the target language
# and reading the tsv file
test_path=os.path.join(path,target_language,"lexicons",target_language+".translit.sampled.test.tsv")
test=pd.read_csv(test_path,sep="\t",names=['tgt','src','count'])




In [ ]:
def one_hot_encoding(src,tgt,s_char,t_char,):
  encoder_tokens=len(s_char)
  decoder_tokens=len(t_char)
  max_s_len=max([len(txt) for txt in src])
  max_t_len=max([len(txt) for txt in tgt])

  # Input to encoder and decoder
  encoder_input=np.zeros((len(src),max_s_len,encoder_tokens),dtype="float32")
  decoder_input=np.zeros((len(src),max_t_len,decoder_tokens),dtype="float32")
  
  # Decoder target
  decoder_target=np.zeros((len(src),max_t_len,decoder_tokens),dtype="float32")

 # Assigning numbers to the characters for the source language 
  source_char_int= dict([(char, i) for i, char in enumerate(s_char)])
  source_int_char= dict((i, char) for char, i in source_char_int.items())
 # Assigning characters to number for the target language
  target_char_int= dict([(char, i) for i, char in enumerate(t_char)])
  target_int_char= dict((i, char) for char, i in source_char_int.items())


  for i,(input_text,target_text) in enumerate(zip(src,tgt)):
    
    # Creating one hot encoding for each character in source language
    for t,char in enumerate(input_text):
      encoder_input[i, t, source_char_int[char]]=1.0
    encoder_input[i, t + 1 :,source_char_int[" "]]=1.0
    
    # Creating one hot encoding for each character in target language
    for t,char in enumerate(target_text):
      decoder_input[ i, t , target_char_int[char]]=1.0
      if t>0:
        decoder_target[i,t-1,target_char_int[char]]=1.0
      decoder_input[i,t+1:,target_char_int[" "]]=1.0
      decoder_target[i,t:,target_char_int[" "]]=1.0
  
  return encoder_input,decoder_input,decoder_target
  

58550


In [ ]:
def data_preprocessing(source,target):
  s_char=set()
  t_char=set()

  source=[str(x) for x in source]
  target=[str(x) for x in target]
  
  s_words=[]
  t_words=[]

  for src,tgt in zip(source,target):
    tgt="\t"+tgt+"\n"
    s_words.append(src)
    t_words.append(tgt)

    # extracting each character from the words and adding them to the respective set
    for char in src:
      if char not in s_char:
        s_char.add(char)
    for char in tgt:
      if char not in t_char:
        t_char.add(char)

  # sorting the characters of source and target languages
  s_char=sorted(list(s_char))
  t_char=sorted(list(t_char))

  # Adding a space
  s_char.append(" ")
  t_char.append(" ")

  return one_hot_encoding(s_words,t_words,s_char,t_char)


In [ ]:
train_source=train["src"].to_list()
train_target=train["tgt"].to_list()

valid_source=valid["src"].to_list()
valid_target=valid["tgt"].to_list()

test_source=test["src"].to_list()
test_target=test["tgt"].to_list()

# Training data
encoder_input_train, decoder_input_train, decoder_target_train = data_preprocessing(train_source,train_target)

# Validation data
encoder_input_valid, decoder_input_valid, decoder_target_valid = data_preprocessing(valid_source,valid_target)

# Test data
encoder_input_test, decoder_input_test, decoder_target_test = data_preprocessing(test_source,test_target)
